In [1]:
import sys; sys.path.append('../src/')
import cloudpickle as cp, numpy as np, pykep as pk
from scipy.optimize import fsolve
from spacecraft import Spacecraft

In [2]:
# get best neural network
nets = cp.load(open('../src/spacecraft_nets.p', "rb"))
#net = nets[np.argmin([net.ltrn[-1] for net in nets])]
net = nets[1]
net

Spacecraft_Controller(
  (0): LayerNorm(torch.Size([8]), eps=1e-05, elementwise_affine=True)
  (1): Linear(in_features=8, out_features=50, bias=True)
  (2): Softplus(beta=1, threshold=20)
  (3): LayerNorm(torch.Size([50]), eps=1e-05, elementwise_affine=True)
  (4): Linear(in_features=50, out_features=50, bias=True)
  (5): Softplus(beta=1, threshold=20)
  (6): LayerNorm(torch.Size([50]), eps=1e-05, elementwise_affine=True)
  (7): Linear(in_features=50, out_features=50, bias=True)
  (8): Softplus(beta=1, threshold=20)
  (9): LayerNorm(torch.Size([50]), eps=1e-05, elementwise_affine=True)
  (10): Linear(in_features=50, out_features=50, bias=True)
  (11): Softplus(beta=1, threshold=20)
  (12): LayerNorm(torch.Size([50]), eps=1e-05, elementwise_affine=True)
  (13): Linear(in_features=50, out_features=3, bias=True)
  (14): Tanh()
)

In [3]:
# get intial state and final time
sc = Spacecraft()
tf = np.load('spacecraft_z_alpha_homotopy_beta_1.npy')[0][0][0]
s0 = np.hstack((*pk.planet.jpl_lp('earth').eph(pk.epoch(0)), sc.M))
sc = Spacecraft()
sc.beta = 0.999
alpha = 0
tl, sl, ul = sc.propagate_controller(tf, s0, net.predict, alpha, atol=1e-13, rtol=1e-13)
sf = np.copy(sl[-1, :])
sf[0:3] /= sc.L
sf[3:6] /= sc.V
sf[6] /= sc.M

In [4]:
net.predict(sf, alpha)

array([ 0.99961673, -0.69494448,  0.71132704,  0.10519506])

In [5]:
def f(x):
    obj = net.predict(x, 0)[0]
    return np.array([obj, 0, 0, 0, 0, 0, 0])

In [27]:
sol= fsolve(f, sf, factor=0.1, epsfcn=1e-10, xtol=1e-12)
sol, net.predict(sol, 0)[0]

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


(array([-1.21181424, -0.34414772, -0.81498759,  1.48121237,  0.08346648,
         0.37511989,  1.16458919]), 2.391954412317432e-10)

In [ ]:
# get intial state and final time
sc = Spacecraft()
tf = np.load('spacecraft_z_alpha_homotopy_beta_1.npy')[0][0][0]
s0 = np.hstack((*pk.planet.jpl_lp('earth').eph(pk.epoch(0)), sc.M))

In [ ]:
def get_eq(net, s0, tf, alpha, beta=0.999):
    
    # instantiate spacecraft
    sc = Spacecraft()
    
    # set dynamics beta paramter
    sc.beta = beta
    
    # get policy trajectory
    tl, sl, ul = sc.propagate_controller(tf, s0, net.predict, alpha, atol=1e-13, rtol=1e-13)
    
    # get final state
    sf = np.copy(sl[-1, :])
    
    # nondimensionalise final state
    sf[0:3] /= sc.L
    sf[3:6] /= sc.V
    sf[6] /= sc.M
    
    # nondimensionalise constants for dynamics
    sc.c1 /= sc.F
    sc.c2 /= sc.Q
    sc.mu /= pk.MU_SUN
    
    # make functions for root solver
    def f(x):
        return sc.ds(x, net.predict(x, alpha))
    def jac(x):
        return sc.dds(x, net.predict(x, alpha))
    
    # solve for root
    sol = root(f, sf, jac=jac, method='hybr', options={'fatol': 1e-8, 'maxiter': 1000, 'disp': True, 'factor': 1, 'jac_options':{'rdiff': }})
    print(sol)
    
    return sol.x, sol.fun, sol.success, sf
    

In [ ]:
x, f, feas, sf = get_eq(net, s0, tf, 0)

In [ ]:
f

In [ ]:
# nondimensionalise final state and dynamics

sc.c1 /= sc.F
sc.c2 /= sc.Q
sc.mu /= pk.MU_SUN

In [ ]:
# create functions for solver
def f(x):
    return sc.ds(x, net.predict(x, 0))
def jac(x):
    return sc.dds(x, net.predict(x, 0))

In [ ]:
sol = root(f, sf, jac=jac, method='krylov', options={'maxiter': 10000, 'disp': True})
sol

In [ ]:
f(sol.x)

In [ ]:
xs = np.copy(sol.x)
xs[0:3] /= sc.L
xs[3:6] /= sc.V
xs[6] /= sc.M
xs

In [ ]:
xs